In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [2]:
!pip install spafe
!pip install praat-parselmouth
!pip install textstat
!pip install pocketsphinx
!pip install ctranslate2==4.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 20.2 MB/s eta 0:00:00


In [3]:
!pip install git+https://github.com/m-bain/whisperx.git


  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-96hudzca
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-96hudzca
  Resolved https://github.com/m-bain/whisperx.git to commit 9e3a9e0e38fcec1304e1784381059a0e2c670be5
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
os.chdir("/content/gdrive/MyDrive/speech_analysis")

In [9]:
from process_file import process_file, process_file_model

In [3]:
import whisperx
import gc
import torch

torch.set_num_threads(1)

vad_model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

device = "cuda"
batch_size = 8 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
transcription_model = whisperx.load_model("large-v3", device, compute_type=compute_type)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:02<00:00, 7.94MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import csv


def calculate_all_features(input_dir, output_file, label):
    previously_calculated = {}
    if os.path.exists(output_file):
        features_pd = pd.read_csv(output_file)
        previously_calculated = set(features_pd["filename"].tolist())


    csvfile = open(output_file, "a+")
    writer = None
    write_header = True
    if len(previously_calculated) > 0:
        write_header = False
        df = pd.read_csv(output_file)
        fieldnames = list(df.columns)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    progress_bar = tqdm(range(len(os.listdir(input_dir))), position=0, leave=True)
    for file in os.listdir(input_dir):
        if file in previously_calculated:
            continue
        if file.endswith('.wav'):
                file_path = os.path.join(input_dir, file)
                try:
                    acoustic_features = process_file(file_path)
                    features = process_file_model(file_path, vad_model, utils, transcription_model)
                    features.update(acoustic_features)
                    # features = {}

                except Exception as e:
                    print(e)
                    print(file)
                    features = {}
                features['filename'] = file
                features['label'] = label
                if write_header:
                    writer = csv.DictWriter(csvfile, fieldnames=features.keys())
                    writer.writeheader()
                    write_header = False
                writer.writerows([features])

        progress_bar.update(1)

    csvfile.close()


In [5]:
directory_path_ad = "/content/gdrive/MyDrive/Data/2021/Audio-denoise_new/test_data_2021/AD-denoise"
directory_path_cn = "/content/gdrive/MyDrive/Data/2021/Audio-denoise_new/test_data_2021/CN-denoise"

In [10]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
calculate_all_features(directory_path_ad, "oo17.csv", 1)
# calculate_all_features(directory_path_cn, "oo1.csv", 0)

  0%|          | 0/35 [00:00<?, ?it/s]

PROCESSED FREQUENCY PARAMETERS...
PROCESSED SPECTRAL DOMAIN...
PROCESSED VOICE QUALITY...
PROCESSED LOUDNESS AND INTENSITY PARAMETERS...
PROCESSED COMPLEXITY...
Frequency parameters: 240


/content/gdrive/.shortcut-targets-by-id/15DWZifyRZ7DWu9FWGZKjeBJhvRx33IHo/speech_analysis/statistical_functions.py:70: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(frame)
/content/gdrive/.shortcut-targets-by-id/15DWZifyRZ7DWu9FWGZKjeBJhvRx33IHo/speech_analysis/statistical_functions.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(frame)


Spectral Domain: 6194
Voice Quality: 6338
Loudness: 6578
Complexity: 6722
6722
Detected language: en (0.99) in first 30s of audio...
alignment_error: (9, 3)
59 78
PROCESSED FREQUENCY PARAMETERS...
PROCESSED SPECTRAL DOMAIN...
PROCESSED VOICE QUALITY...
PROCESSED LOUDNESS AND INTENSITY PARAMETERS...
PROCESSED COMPLEXITY...
Frequency parameters: 240


/content/gdrive/.shortcut-targets-by-id/15DWZifyRZ7DWu9FWGZKjeBJhvRx33IHo/speech_analysis/statistical_functions.py:70: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(frame)
/content/gdrive/.shortcut-targets-by-id/15DWZifyRZ7DWu9FWGZKjeBJhvRx33IHo/speech_analysis/statistical_functions.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(frame)


Spectral Domain: 6194
Voice Quality: 6338
Loudness: 6578
Complexity: 6722
6722
Detected language: en (1.00) in first 30s of audio...
alignment_error: (8, 1)
59 78


In [20]:
import pandas as pd

df = pd.read_csv("oo17.csv")
df = df.dropna(axis=1, how='all')

print(f"Dimension: {df.shape}")


Dimension: (2, 6848)
